- Started on July 31, 2019
- Works with a subset of the highd dataset
- Goal is to run particle filtering on this highd dataset with the
ultimate aim of learning idm rules for driver modeling
- Trivia: Timestep is 0.04 sec, each vehicle is visible for median duration of
13.6 seconds. This translates to about 340 timesteps of visibility. We have verified
this by starting from scene 1 and going till 340 looking at truck vehicle id 27

## Needs experimentation
- Should the particle fitness be based on likelihood of hallucinated x location
under Gaussian centered around true x location? Or should this also include
y? Reason for asking this question is because Frenet s is 0 all the time so
we will need to work with posG i.e. global position

## Needs investigation
- Why is Frenet s always 0?

# Import packages, load data, include helpers

In [59]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using PyPlot # For in notebook plotting
using Distributions
using Test
using StatsBase # For the weights function in likelihood sampling

In [5]:
# The trajectories i.e. positions of the cars with timestep
traj_highd = open(io->read(io, MIME"text/plain"(), Trajdata), 
    "trajdata_Data11_Road1_nLIOD3_lower.txt", "r");
# The highd roadway geometry
roadway_highd = open(io->read(io, MIME"text/plain"(), Roadway), 
    "highd_roadway.txt", "r");

In [46]:
include("admin_functions.jl");

# Workspace

In [67]:
scene = Scene(500)
get!(scene,traj_highd,2)
# We are focusing on vehicle id 3 hence the findfirst(3,scene)
trupos = scene.entities[findfirst(3,scene)].state.posG.x
@show trupos

p_set_dict = gen_test_particles(50)
@show p_set_dict
p_set_new = update_p_one_step(roadway,scene,trupos,p_set_dict,car_id=3)

trupos = 298.77000956064006
p_set_dict = Dict(:v_des=>[17.0, 11.0, 22.0, 26.0, 24.0, 19.0, 18.0, 28.0, 13.0, 23.0, 11.0, 17.0, 20.0, 26.0, 23.0, 30.0, 26.0, 16.0, 29.0, 29.0, 15.0, 15.0, 20.0, 16.0, 26.0, 21.0, 27.0, 24.0, 12.0, 28.0, 16.0, 12.0, 15.0, 10.0, 27.0, 25.0, 18.0, 12.0, 16.0, 17.0, 17.0, 30.0, 19.0, 14.0, 10.0, 20.0, 18.0, 10.0, 22.0, 16.0],:σ=>[0.7, 0.7, 0.5, 0.2, 1.0, 0.9, 1.0, 0.4, 0.2, 0.2, 0.5, 1.0, 0.9, 1.0, 0.6, 0.3, 0.3, 0.5, 0.4, 0.7, 0.3, 0.6, 0.5, 0.5, 0.8, 0.8, 0.4, 0.6, 1.0, 1.0, 1.0, 0.2, 0.4, 0.5, 0.5, 0.4, 0.1, 0.6, 0.7, 0.6, 0.4, 0.7, 0.1, 0.4, 0.1, 1.0, 0.7, 0.9, 0.3, 0.1])


Dict{Any,Any} with 2 entries:
  :v_des => [17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0  …  17.…
  :σ     => [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7  …  0.7, 0.7, 0.7…

In [60]:
function update_p_one_step(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Provide valid car_id" end
    
    lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,f,trupos,p_set_dict,car_id=car_id)
    
    num_params = size(p_mat)[1]
    num_p = size(p_mat)[2]
    
    p_weight_vec = weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
    idx = sample(1:num_p,p_weight_vec,num_p)
    new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
        
    new_p_set_dict = to_dict_form(params,new_p_mat)
    return new_p_set_dict
end

update_p_one_step (generic function with 1 method)

# Filtering functions

In [ ]:
function compute_particle_likelihoods(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    timestep = 0.04 #TODO: Remove hardcoding, 0.04 is for highd
    p_mat, params, vec_val_vec = to_matrix_form(p_set_dict)
    
    num_params=size(p_mat)[1]
    num_p = size(p_mat)[2]
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p    
        # Create dict version for a single particle
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        
        # TODO: This math needs to be verified from random variable calculations
        std_dev_pos = timestep*timestep*std_dev_acc
            
        hpos = hallucinate_a_step(roadway,f,p_dict,car_id=car_id)
        lkhd_vec[i] = pdf(Normal(hpos,std_dev_pos),trupos[1])
    end
    return lkhd_vec,p_mat,params
end

In [ ]:
# Test for compute_particle_likelihoods
v_array = [10.,15.,20.,25.,30.]
num_p = length(v_array)
sig_array = [0.1,0.1,0.1,0.1,0.1]	
p_set_dict = Dict(:v_des=>v_array,:σ=>sig_array)
td1 = traj_highd;
scene = Scene(500)
get!(scene,td1,1)
roadway = roadway_highd;
trupos = hallucinate_a_step(roadway,scene,Dict(:v_des=>25.0,:σ=>0.0),car_id=3)
lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,scene,trupos,p_set_dict,car_id=3)

@test length(lkhd_vec) == num_p
@test length(params) == 2
@test size(p_mat)[1] == 2
@test size(p_mat)[2] == 5
@test any(isnan,lkhd_vec) == false #Gets triggered when wrong car_id called
# For example only 1 car on road but you say car_id = 2

# Driving functions

In [51]:
"""
# Example:
scene = Scene(500)
get!(scene,traj_highd,1)
vehid = scene.entities[2].id
@show vehid
vehicle = scene.entities[findfirst(scene.entities[2].id,scene)]
print("Location before hallucination = $(vehicle.state.posG.x)\n")
particle = Dict(:v_des=>25.0,:σ=>0.5)
hall_pos = hallucinate_a_step(roadway_highd,scene,
    particle,car_id=scene.entities[2].id)
print("Hallucinated position = $(hall_pos[1])\n")
scene = Scene(500)
get!(scene,traj_highd,2)
vehnew = scene.entities[findfirst(vehid,scene)]
print("True position in timestep 2 = $(vehnew.state.posG.x)")
"""
function hallucinate_a_step(roadway,scene_input,particle;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    
    scene = deepcopy(scene_input)
    #scene = scene_input # This was the failure case

    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for veh in scene
        if veh.id == car_id
            models[veh.id] = IntelligentDriverModel(;particle...)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles
            models[veh.id] = IntelligentDriverModel(v_des=10.0)
        end
    end
    
    n_steps = 1
    dt = 0.04
    rec = SceneRecord(n_steps, dt)
    
    simulate!(rec, scene, roadway, models, n_steps)
    
    X = Array{Float64}(undef,n_steps, 1)

    for t in 1:n_steps
        f = rec.frames[n_steps - t + 1]
        
            # Access the vehicle with id as car_id and return its frenet s
        X[t,1] = scene.entities[findfirst(car_id,f)].state.posG.x
    end
    return X[1]
end

hallucinate_a_step

In [ ]:
# Test for hallucinate_a_step
scene = Scene(500)
get!(scene,traj_highd,1)
vehid = scene.entities[2].id
@show vehid
vehicle = scene.entities[findfirst(scene.entities[2].id,scene)]
print("Location before hallucination = $(vehicle.state.posG.x)\n")
particle = Dict(:v_des=>25.0,:σ=>0.5)
hall_pos = hallucinate_a_step(roadway_highd,scene,
    particle,car_id=scene.entities[2].id)
print("Hallucinated position = $(hall_pos[1])\n")
scene = Scene(500)
get!(scene,traj_highd,2)
vehnew = scene.entities[findfirst(vehid,scene)]
print("True position in timestep 2 = $(vehnew.state.posG.x)")

# Visualization functions

In [31]:
"""
# Example: make_video(range=1:20,filename="test.mp4")
"""
function make_video(;range=nothing,filename="output.mp4")

    frames = Frames(MIME("image/png"), fps=10)
    scene = Scene(500)
    for i in range
        temp_scene = get!(scene,traj_highd,i)

        overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
        pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene]
        
        scene_visual = render(temp_scene, roadway_highd, 
            #cam=CarFollowCamera{Int}(2,5.0),
            #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
            cam=FitToContentCamera(0.),
            overlays)
        push!(frames,scene_visual)
    end
    write(string("media/"*filename),frames)    
    
    return nothing
end

make_video (generic function with 1 method)

In [ ]:
# For overlaying hallucination on the ground truth
# Borrowed from visualization_functions.jl
struct my_overlay <: SceneOverlay
    scene::Scene
    color # Needs to be of form colorant"Colorname"
end

"""
Render method for `my_overlay`. Helpful for making the color choice
"""
function AutoViz.render!(rendermodel::RenderModel,overlay::my_overlay, 
        scene::Scene, roadway::Roadway)
    AutoViz.render!(rendermodel,overlay.scene,car_color = overlay.color)
    return rendermodel
end

# Code dump for use later/make functions

In [ ]:
# Drive using IDM
scene = Scene(500)
temp = get!(scene,traj_highd,1)
models = Dict{Int,DriverModel}()
for veh in scene
    models[veh.id] = IntelligentDriverModel(v_des=40.0)
end
n_steps = 100
dt = 0.04 # To match the timestep of traj_highd, ngsim was timestep 0.1
rec = SceneRecord(n_steps,dt)
simulate!(rec,scene,roadway_highd,models,n_steps)

In [ ]:
# highd replays with idm hallucination overlayed
frames = Frames(MIME("image/png"), fps=10)
scene = Scene(500)
for i in 1:100
    temp_scene = get!(scene,traj_highd,i)

    overlay_hallucination = my_overlay(rec[i-100],colorant"white")
    
    scene_visual = render(temp_scene, roadway_highd, 
        #cam=CarFollowCamera{Int}(2,5.0),
        #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
        cam=FitToContentCamera(0.),
        [overlay_hallucination])
    push!(frames,scene_visual)
end
write("media/highd_halluc_vdes40.mp4",frames)